## **Procesamiento y consumo incremental de datos con Spark Structured Streaming**

### **Consumo de datos hacia sistema local**

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import year, month, dayofmonth, lpad
from pyspark.sql.functions import to_date
import requests
import os
import subprocess

spark = SparkSession. \
    builder. \
    config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1'). \
    config('spark.sql.warehouse.dir', '/user/local/spark/warehouse'). \
    config('spark.master', 'local[*]'). \
    enableHiveSupport(). \
    appName('Consumo y procesamiento de datos con Spark Structured Streaming'). \
    getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d835932-9f62-439a-844d-ff0406cb2d10;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.1 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 3743ms :: artifacts dl 225ms
	:: modules in use:
	com.

In [2]:
def upload_gharchive_files_to_hdfs(file_name):
    year = file_name[:4]
    month = file_name[5:7]
    day = file_name[8:10]
    
    file_url = f'https://raw.githubusercontent.com/perezlino/data_fake/main/{file_name}'
    
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        
        target_local_folder = f'/spark_streaming/data/ghactivity/anio={year}/mes={month}/dia={day}'
        os.makedirs(target_local_folder, exist_ok=True)

        local_file_path = os.path.join(target_local_folder, file_name)
        with open(local_file_path, 'w', newline='', encoding='utf-8') as target_file:
            target_file.write(response.text)
        
        target_hdfs_folder = f'/proyecto/spark/streaming/landing/ghactivity/anio={year}/mes={month}/dia={day}'
        subprocess.check_call(f'hdfs dfs -mkdir -p {target_hdfs_folder}', shell=True)
        subprocess.check_call(f'hdfs dfs -put {local_file_path} {target_hdfs_folder}', shell=True)
    
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo: {e}")
    except subprocess.CalledProcessError as e:
        print(f"Error al ejecutar el comando: {e}")
    except Exception as e:
        print(f"Error inesperado: {e}")

### **Procesamiento y carga de datos para el 2024-10-08 en HDFS**

In [6]:
file_date = '2024-10-09'

In [7]:
def processing_files(file_date):

    for hour in range(0, 6):
        print (f'Procesando archivo {file_date}-{hour}.csv')
        upload_gharchive_files_to_hdfs (f'{file_date}-{hour}.csv')

In [8]:
processing_files(file_date)

Procesando archivo 2024-10-09-0.csv
Procesando archivo 2024-10-09-1.csv
Procesando archivo 2024-10-09-2.csv
Procesando archivo 2024-10-09-3.csv
Procesando archivo 2024-10-09-4.csv
Procesando archivo 2024-10-09-5.csv


In [9]:
# Establecer el número de particiones de mezcla
spark.conf.set("spark.sql.shuffle.partitions", 16)

In [ ]:
# Tener en cuenta que, por lo general, no inferimos el schema, ya que se desperdiciaría capacidad de cómputo al escanear los datos 
# solo para inferir el schema. En su lugar, aplicamos el schema manualmente.
# No lo usaremos
spark.conf.set('spark.sql.streaming.schemaInference', 'true')

In [10]:
# Con cleanSource='archive', los archivos procesados se archivaran en la siguiente ejecución del micro-batch
# Cuando utilizamos 'archive', también necesitamos establecer el directorio de archivación utilizando `sourceArchiveDir`
# Con maxFilePerTrigger=3, en cada micro-batch solo se procesaran 3 archivos
# maxFilesPerTrigger no es aplicable cuando se ejecutan trabajos utilizando trigger (once=True)
# Al ser 6 archivos y maxFilePerTrigger=3, se generaran 2 jobs. El primer job comenzará con los 3 primeros archivos 
# '2024-10-08-0.csv', '2024-10-08-1.csv' y '2024-10-08-2.csv'. Y en el segundo Batch se procesan los 3 archivos restantes 
# '2024-10-08-3.csv', '2024-10-08-4.csv' y '2024-10-08-5.csv'.
# En este script se ejecutaron 2 micro-batch (2 jobs), al ejecutarse el segundo job (última ejecución), es en este momento donde 
# se activa cleanSource='archive' y archiva todo lo que encuentre de ejecuciones anteriores del directorio /landing/ghactivity, asi 
# archivando tanto los archivos subidos por el segundo job realizado en el script 'carga_2024_10_08.ipynb' y los 3 primeros archivos 
# subidos en este script (1er job)  '2024-10-09-0.csv', '2024-10-08-9.csv' y '2024-10-08-9.csv'
# Realizamos la definición del schema, dado que en .readStream no se permite inferir el schema en un archivo CSV

schema = StructType([
    StructField("Nombre", StringType(), True),
    StructField("Apellido", StringType(), True),
    StructField("Edad", IntegerType(), True),
    StructField("Ciudad", StringType(), True),
    StructField("Trabajo", StringType(), True),
    StructField("Telefono", StringType(), True),
    StructField("Fecha", StringType(), True)
])

ghactivity_df = spark. \
    readStream. \
    format('csv'). \
    option('maxFilesPerTrigger', 3). \
    option('cleanSource', 'archive'). \
    option('sourceArchiveDir', '/proyecto/spark/streaming/landing/archive/ghactivity'). \
    option("header", "true"). \
    option("delimiter", ","). \
    schema(schema). \
    load ('/proyecto/spark/streaming/landing/ghactivity')

In [11]:
ghactivity_df.isStreaming

True

In [12]:
ghactivity_df.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Apellido: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Trabajo: string (nullable = true)
 |-- Telefono: string (nullable = true)
 |-- Fecha: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)



In [13]:
# Debemos utilizar las mismas ubicaciones que utilizamos para todas las cargas tanto para 'path' como para 'checkpoint'. 
ghactivity_df. \
  writeStream. \
  format('parquet'). \
  partitionBy('anio', 'mes', 'dia'). \
  option("checkpointLocation", "/proyecto/spark/streaming/bronze/checkpoint/ghactivity"). \
  option("path", "/proyecto/spark/streaming/bronze/data/ghactivity"). \
  trigger(processingTime = '60 seconds'). \
  start()

2024-10-11 19:00:04,645 WARN hdfs.DFSClient: Caught exception       (0 + 3) / 3]
java.lang.InterruptedException
	at java.lang.Object.wait(Native Method)
	at java.lang.Thread.join(Thread.java:1257)
	at java.lang.Thread.join(Thread.java:1331)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.closeResponder(DFSOutputStream.java:716)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.endBlock(DFSOutputStream.java:476)
	at org.apache.hadoop.hdfs.DFSOutputStream$DataStreamer.run(DFSOutputStream.java:652)


### **Verificación de datos consumidos y procesados**

In [14]:
# Validación de la ubicación de los datos para ver si los archivos de la última ejecución anterior fueron archivados. 
# Dado que se utilizo cleanSource='archive' en este job, se archivaran los archivos de la última ejecución que fue realizada 
# antes de realizar la ejecución actual. 
# Si deberias ver los archivos relacionados con la ejecución anterior
!hdfs dfs -ls -R /proyecto/spark/streaming/landing/archive/ghactivity

drwxr-xr-x   - root supergroup          0 2024-10-11 18:59 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto
drwxr-xr-x   - root supergroup          0 2024-10-11 18:59 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto/spark
drwxr-xr-x   - root supergroup          0 2024-10-11 18:59 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto/spark/streaming
drwxr-xr-x   - root supergroup          0 2024-10-11 18:59 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto/spark/streaming/landing
drwxr-xr-x   - root supergroup          0 2024-10-11 18:59 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto/spark/streaming/landing/ghactivity
drwxr-xr-x   - root supergroup          0 2024-10-11 18:59 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto/spark/streaming/landing/ghactivity/anio=2024
drwxr-xr-x   - root supergroup          0 2024-10-11 19:00 /proyecto/spark/streaming/landing/archive/ghactivity/proyecto/spark/streaming/landing/gha

In [17]:
!hdfs dfs -ls /proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08

In [18]:
!hdfs dfs -ls /proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09

Found 3 items
-rw-r--r--   3 root supergroup      12015 2024-10-11 18:56 /proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-3.csv
-rw-r--r--   3 root supergroup      12025 2024-10-11 18:57 /proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-4.csv
-rw-r--r--   3 root supergroup      11999 2024-10-11 18:57 /proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-5.csv


In [19]:
!hdfs dfs -ls /proyecto/spark/streaming/bronze/checkpoint/ghactivity

Found 4 items
drwxr-xr-x   - root supergroup          0 2024-10-11 19:00 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/commits
-rw-r--r--   3 root supergroup         45 2024-10-11 18:04 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/metadata
drwxr-xr-x   - root supergroup          0 2024-10-11 19:00 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets
drwxr-xr-x   - root supergroup          0 2024-10-11 18:04 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources


In [20]:
!hdfs dfs -ls -R /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources

drwxr-xr-x   - root supergroup          0 2024-10-11 19:00 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0
-rw-r--r--   3 root supergroup        932 2024-10-11 18:04 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/0
-rw-r--r--   3 root supergroup        932 2024-10-11 18:21 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/1
-rw-r--r--   3 root supergroup        467 2024-10-11 18:36 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/2
-rw-r--r--   3 root supergroup        467 2024-10-11 18:37 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/3
-rw-r--r--   3 root supergroup        467 2024-10-11 18:59 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/4
-rw-r--r--   3 root supergroup        467 2024-10-11 19:00 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/5


In [21]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/0

v1
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=06/2024-10-06-0.csv","timestamp":1728669535124,"batchId":0}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=06/2024-10-06-1.csv","timestamp":1728669565158,"batchId":0}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=06/2024-10-06-2.csv","timestamp":1728669583832,"batchId":0}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=06/2024-10-06-3.csv","timestamp":1728669607100,"batchId":0}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=06/2024-10-06-4.csv","timestamp":1728669620971,"batchId":0}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=06/2024-10-06-5.csv","timestamp":1728669635791,"batchId":0}

In [22]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/1

v1
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=07/2024-10-07-0.csv","timestamp":1728670749696,"batchId":1}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=07/2024-10-07-1.csv","timestamp":1728670767999,"batchId":1}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=07/2024-10-07-2.csv","timestamp":1728670780998,"batchId":1}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=07/2024-10-07-3.csv","timestamp":1728670797594,"batchId":1}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=07/2024-10-07-4.csv","timestamp":1728670810378,"batchId":1}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=07/2024-10-07-5.csv","timestamp":1728670823733,"batchId":1}

In [23]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/2

v1
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08/2024-10-08-5.csv","timestamp":1728671729311,"batchId":2}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08/2024-10-08-4.csv","timestamp":1728671717202,"batchId":2}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08/2024-10-08-3.csv","timestamp":1728671702934,"batchId":2}

In [24]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/3

v1
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08/2024-10-08-2.csv","timestamp":1728671689910,"batchId":3}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08/2024-10-08-1.csv","timestamp":1728671677134,"batchId":3}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=08/2024-10-08-0.csv","timestamp":1728671664544,"batchId":3}

In [25]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/4

v1
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-0.csv","timestamp":1728672938335,"batchId":4}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-1.csv","timestamp":1728672956752,"batchId":4}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-2.csv","timestamp":1728672969693,"batchId":4}

In [26]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/sources/0/5

v1
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-3.csv","timestamp":1728673006982,"batchId":5}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-4.csv","timestamp":1728673026351,"batchId":5}
{"path":"hdfs://namenode:9000/proyecto/spark/streaming/landing/ghactivity/anio=2024/mes=10/dia=09/2024-10-09-5.csv","timestamp":1728673039786,"batchId":5}

In [27]:
!hdfs dfs -ls /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets

Found 6 items
-rw-r--r--   3 root supergroup        471 2024-10-11 18:04 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/0
-rw-r--r--   3 root supergroup        471 2024-10-11 18:21 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/1
-rw-r--r--   3 root supergroup        471 2024-10-11 18:36 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/2
-rw-r--r--   3 root supergroup        471 2024-10-11 18:37 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/3
-rw-r--r--   3 root supergroup        471 2024-10-11 18:59 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/4
-rw-r--r--   3 root supergroup        471 2024-10-11 19:00 /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/5


In [28]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/0

v1
{"batchWatermarkMs":0,"batchTimestampMs":1728669871299,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":0}

In [29]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/1

v1
{"batchWatermarkMs":0,"batchTimestampMs":1728670861090,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":1}

In [30]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/2

v1
{"batchWatermarkMs":0,"batchTimestampMs":1728671762892,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":2}

In [31]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/3

v1
{"batchWatermarkMs":0,"batchTimestampMs":1728671820511,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":3}

In [32]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/4

v1
{"batchWatermarkMs":0,"batchTimestampMs":1728673192670,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":4}

In [33]:
!hdfs dfs -cat /proyecto/spark/streaming/bronze/checkpoint/ghactivity/offsets/5

v1
{"batchWatermarkMs":0,"batchTimestampMs":1728673203112,"conf":{"spark.sql.streaming.stateStore.providerClass":"org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider","spark.sql.streaming.join.stateFormatVersion":"2","spark.sql.streaming.flatMapGroupsWithState.stateFormatVersion":"2","spark.sql.streaming.multipleWatermarkPolicy":"min","spark.sql.streaming.aggregation.stateFormatVersion":"2","spark.sql.shuffle.partitions":"16"}}
{"logOffset":5}

In [34]:
!hdfs dfs -ls -R /proyecto/spark/streaming/bronze/data/ghactivity

drwxr-xr-x   - root supergroup          0 2024-10-11 19:00 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata
-rw-r--r--   3 root supergroup        866 2024-10-11 18:04 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata/0
-rw-r--r--   3 root supergroup        866 2024-10-11 18:21 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata/1
-rw-r--r--   3 root supergroup        866 2024-10-11 18:36 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata/2
-rw-r--r--   3 root supergroup        866 2024-10-11 18:37 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata/3
-rw-r--r--   3 root supergroup        866 2024-10-11 19:00 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata/4
-rw-r--r--   3 root supergroup        866 2024-10-11 19:00 /proyecto/spark/streaming/bronze/data/ghactivity/_spark_metadata/5
drwxr-xr-x   - root supergroup          0 2024-10-11 18:04 /proyecto/spark/streaming/bronze/data/ghactivity/anio=2024
dr

### **Validación de los datos consumidos**

In [35]:
!hdfs dfs -ls /proyecto/spark/streaming/bronze/data/ghactivity/anio=2024/mes=10

Found 4 items
drwxr-xr-x   - root supergroup          0 2024-10-11 18:04 /proyecto/spark/streaming/bronze/data/ghactivity/anio=2024/mes=10/dia=6
drwxr-xr-x   - root supergroup          0 2024-10-11 18:21 /proyecto/spark/streaming/bronze/data/ghactivity/anio=2024/mes=10/dia=7
drwxr-xr-x   - root supergroup          0 2024-10-11 18:37 /proyecto/spark/streaming/bronze/data/ghactivity/anio=2024/mes=10/dia=8
drwxr-xr-x   - root supergroup          0 2024-10-11 19:00 /proyecto/spark/streaming/bronze/data/ghactivity/anio=2024/mes=10/dia=9


In [36]:
ghactivity = spark. \
    read. \
    parquet(f'/proyecto/spark/streaming/bronze/data/ghactivity')

In [37]:
ghactivity.count()

4800

In [38]:
ghactivity.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Apellido: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Trabajo: string (nullable = true)
 |-- Telefono: string (nullable = true)
 |-- Fecha: string (nullable = true)
 |-- anio: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- dia: integer (nullable = true)



In [39]:
# Ahora estoy tratando de obtener el recuento para 'Ingenieros' que trabajen en 'Madrid'. Ahora tenemos datos de cuatro dias.
ghactivity. \
    filter("Ciudad = 'Madrid' AND Trabajo = 'Ingeniero' "). \
    count()

166

In [40]:
# Ahora estoy agrupando por fecha y luego obtengo el recuento. Aquí obtenemos el recuento total de tres dias.
ghactivity. \
    groupBy('ciudad'). \
    count(). \
    show()

+---------+-----+
|   ciudad|count|
+---------+-----+
|Barcelona| 1034|
|   Madrid|  936|
| Valencia|  968|
|  Sevilla|  912|
|   Bilbao|  950|
+---------+-----+



In [41]:
ghactivity. \
    groupby('anio', 'mes', 'dia'). \
    count(). \
    show()

+----+---+---+-----+
|anio|mes|dia|count|
+----+---+---+-----+
|2024| 10|  8| 1200|
|2024| 10|  6| 1200|
|2024| 10|  7| 1200|
|2024| 10|  9| 1200|
+----+---+---+-----+



In [42]:
# Esto realmente da un recuento diario de 'Ingenieros' que trabajen en 'Madrid'. Ahora tenemos datos de cuatro dias.
ghactivity. \
    filter("Ciudad = 'Madrid' AND Trabajo = 'Ingeniero' "). \
    groupby('anio', 'mes', 'dia'). \
    count(). \
    show()

+----+---+---+-----+
|anio|mes|dia|count|
+----+---+---+-----+
|2024| 10|  8|   37|
|2024| 10|  6|   42|
|2024| 10|  7|   45|
|2024| 10|  9|   42|
+----+---+---+-----+

